In [ ]:
#cells will fill entire width of the browser
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

#Tells Jupyter to reload custom classes from scratch everytime an import cell is run, if you edit a custom class
#between imports Jupyter would otherwise need to be restarted completely. Buyer beware: old class objects in the 
#current namespace will cause errors at execution
%load_ext autoreload
%autoreload 2

#switches matplotlib to show plots in the browser rather than opening a new window
%matplotlib inline

#always forget to do this for better looking plots
import seaborn
seaborn.set()

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import datetime
from cvxpy import *
from statsmodels.tsa import stattools
from sklearn import preprocessing
from sklearn.linear_model import Ridge
import random
import copy
import scipy
import sklearn.metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

In [ ]:
class linear_nnet(nn.Module):
    #linear model for kernelized inputs
    def __init__(self, params):
        super(linear_nnet, self).__init__()
        self.D_in = params['FEATURE_DIM']
        self.D_out = params['OUTPUT_DIM']
        self.l1 = nn.Linear(self.D_in, self.D_out)
    
    def forward(self, x):
        x = self.l1(x) #linear weights for model interpretability
        return(x)

In [ ]:
#Bayesian classifier
def mat_C(x):
    #x shape = px1
    C = np.dot(x, x.T) + np.eye(x.shape[0])
    return(C)

def mat_D(x, y, A):
    #x shape px1, y shape nx1, A nxp
    D = np.dot(y, x.T) + A
    return(D)

def likelihood_point(x, y, A):
    C = mat_C(x)
    D = mat_D(x, y, A)
    const = x.shape[0]*np.log(np.linalg.det(np.linalg.inv(C)))
    var = np.trace(y.dot(y.T) - 2*A.dot(x).dot(y.T) + x.T.dot(A.T).dot(A).dot(x))
    var2 = np.trace(y.dot(y.T) + A.dot(A.T) - D.dot(np.linalg.inv(C)).dot(D.T))   #mixed term doesn't look right
    out = var - var2 + const
    return(out)

In [ ]:
#features to ignore"
ignore = ["Performance Curve Input Variable 1 Value [] !Each Call", "Performance Curve Output Value [] !Each Call"]

#def parse_line()

def read_data_dictionary(lines):
    data_dict = {}
    for line in lines:
        l = line.strip()
        if l == "End of Data Dictionary":
            break
        else:
            tokens = l.split(",")
            if int(tokens[0]) == 2:
                data_dict[int(tokens[0])] = tokens[1:]
                #parse time dictionary line
            elif int(tokens[0]) > 6 and int(tokens[1]) == 1:
                if tokens[-1] in ignore:
                    pass
                elif len(tokens) == 4:
                    data_dict[int(tokens[0])] = tokens[-2] + tokens[-1]
                    
                else:
                    #parse value dictionary line
                    data_dict[int(tokens[0])] = tokens[-1]
            else:
                pass
    return(data_dict)

def parse_time_line(line_str):
    #2,8,Day of Simulation[],Month[],Day of Month[],DST Indicator[1=yes 0=no],Hour[],StartMinute[],EndMinute[],DayType
    #2, 1, 1, 1, 0, 1, 0.00, 2.00, Holiday
    #only want hourly times, multiple minute 2's, assuming got correct
    tokens = line_str.strip().split(",")
    minute=float(tokens[-3])
    hour=int(tokens[-4])
    day=int(tokens[-6])
    month=int(tokens[-7])
    return(datetime.datetime(2017, month, day, hour-1, 0, 0)) #2017 so that the 1st is a Sunday + not leapyear
    #return(str(month) + "/" + str(day) + " " + str(hour) + ":00")
     
def parse_lines(lines, feature_inds):
    data_values = {}
    data_values[2] = []
    for i in feature_inds:
        data_values[i] = []
    
    start_i = lines.index("End of Data Dictionary\n")
    for line in lines[start_i+1:-2]:
        tokens = line.strip().split(",")
        if int(tokens[0]) == 2:
            if float(tokens[-3]) == 0.00 and float(tokens[-2]) == 60.00:
                t = parse_time_line(line)
                data_values[2].append(t)
        elif int(tokens[0]) in feature_inds:
            data_values[int(tokens[0])].append(float(tokens[-1]))
        else:
            pass
    return(data_values)
    #return hourly list for all feature vars, plus hourly list with times to check sorting
    
def get_holidays(lines):
    holidays = []
    weekends = []
    all_dates = []
    
    start_i = lines.index("End of Data Dictionary\n")
    for line in lines[start_i+1:-2]:
        tokens = line.strip().split(",")
        if tokens[-1] == "Holiday":
            if float(tokens[-3]) == 0.00 and float(tokens[-2]) == 60.00:
                t = parse_time_line(line)
                holidays.append(t)
        elif int(tokens[0]) == 2:
            if float(tokens[-3]) == 0.00 and float(tokens[-2]) == 60.00:
                t = parse_time_line(line)
                all_dates.append(t)
                if t.weekday() == 6 or t.weekday() == 0:
                    weekends.append(t)
    return(all_dates, weekends, holidays)
                      
def norm_array(arr):
    return(preprocessing.minmax_scale(arr, axis=1))

def read_total_data_array(filepath, dep_var_names, ind_var_names, zone_temp_inds, first_diff=False, no_weekend_holiday=False):
    with open(datapath + filepath) as d:
        lines = d.readlines()
        lines.pop(0)
        data_dict = read_data_dictionary(lines)
        data = parse_lines(lines, data_dict)
        all_dates, weekends, holidays = get_holidays(lines)
        ind_map = {}
        for key, value in data_dict.items():
            if type(value) == list:
                value = value[-1]
            ind_map[value] = key

    X = np.zeros((len(dep_var_names) + len(zone_temp_inds), 8760))
    U = np.zeros((len(ind_var_names), 8760))

    for i in range(len(ind_var_names)):
        item = ind_var_names[i]
        ind = ind_map[item]
        if item == 'DayType':
            lis = data[ind]
            out = []
            for val in lis:
                out.append(val.weekday())
            arr = np.array(out)
        else:
            arr = np.array(data[ind])
        U[i,:] = arr

    for j in range(len(dep_var_names)):
        item = dep_var_names[j]
        ind = ind_map[item]
        arr = np.array(data[ind])
        X[j,:] = arr

    for j in range(len(zone_temp_inds)):
        ind = zone_temp_inds[j]
        arr = np.array(data[ind])
        X[j+len(dep_var_names),:] = arr
        
    if no_weekend_holiday == True:
        weekend_i = []
        holiday_i = []
        for j in range(len(all_dates)):
            if all_dates[j] in weekends:
                weekend_i.append(j)
            if all_dates[j] in holidays:
                holiday_i.append(j)
                
        del_i = np.array(list(set(holiday_i + weekend_i)))
        X = np.delete(X, del_i, axis=1)
        U = np.delete(U, del_i, axis=1)
    
    if first_diff == True:
        X = np.diff(X, n=1, axis=1)
        U = np.diff(U, n=1, axis=1)

    Z = np.vstack((X, U))

    return(Z, X, U)

def minibatch_X_Y_arrays(X_arr, Y_arr, batchsize):    
    #list of training, target pair tuples
    remainder = X_arr.shape[1] % batchsize
    diff = batchsize - remainder
    tail_X = X_arr[:,-diff:] 
    tail_Y = Y_arr[:,-diff:]
    out_X = [ X_arr[:,i*batchsize:(i+1)*batchsize] for i in range(int(float(X_arr.shape[1])/float(batchsize))) ]
    out_Y = [ Y_arr[:,i*batchsize:(i+1)*batchsize] for i in range(int(float(Y_arr.shape[1])/float(batchsize)))]
    out_X = out_X + [tail_X]
    out_Y = out_Y + [tail_Y]
    return(out_X, out_Y)

def split_train_val(Z, X, U, order=1, shuff=True, split_prop=0.8):
    all_pairs = []
    for i in range(Z.shape[1] - order):  #not 8760 for first-diff
        all_pairs.append((Z[:,i:i+order-1].flatten(), Z[0:-U.shape[0],i+order]))

    if shuff == True:
        random.shuffle(all_pairs) 
    else:
        pass

    train_pairs = all_pairs[0:int(split_prop*len(all_pairs))]
    val_pairs = all_pairs[-int((1.0-split_prop)*len(all_pairs)):]
    
    #normalize to training data
    X_train = []
    Y_train = []

    for item in train_pairs:
        X_train.append(item[0])
        Y_train.append(item[1])

    X_val = []
    Y_val = []
    for item in val_pairs:
        X_val.append(item[0])
        Y_val.append(item[1])

    X_train = np.asarray(X_train).T
    Y_train = np.asarray(Y_train).T

    X_val = np.asarray(X_val).T
    Y_val = np.asarray(Y_val).T
    
    return(X_train, Y_train, X_val, Y_val)

def normalize_data_set(X, Y, X_minmax_vals):
    X_norm = copy.copy(X)
    Y_norm = copy.copy(Y)
    
    for row in range(X.shape[0]):
        mx = np.max(X_minmax_vals[row,:])
        mn = np.min(X_minmax_vals[row,:])

        X_norm[row,:] = (1.0/(mx - mn))*(X[row,:] - mn)

        if row < Y.shape[0]:
            Y_norm[row,:] = (1.0/(mx - mn))*(Y[row,:] - mn)
            
    return(X_norm, Y_norm)

def polynomial_kernel_mat(X, poly_degree):
    out = X
    for d in range(2,poly_degree+1):
        out = np.vstack((out, np.power(X, d)))
    return(out)

In [ ]:
#global vars
datapath = "/home/chase/projects/building_transfer/data/eplus_sims/"

#shared simulation var names
#dep_var_names = ['Electricity:Facility [J] !Hourly', 'Fans:Electricity [J] !Hourly', 'Cooling:Electricity [J] !Hourly']
dep_var_names = ['Fans:Electricity [J] !Hourly']
ind_var_names = ['EnvironmentSite Outdoor Air Drybulb Temperature [C] !Hourly', 'EnvironmentSite Outdoor Air Relative Humidity [%] !Hourly']#, 'DayType']

zone_temp_inds = [863,864,865]

In [ ]:
poly_degrees = [2,3,4,5]
orders = [4,5,6,7,8,9,10,11,12]
lags = [10,20,30,40,50,60,70,80,90,100,110,120,130,140]

prec_values = {}
rec_values = {}

In [ ]:
for poly_degree in poly_degrees:
    prec_values[poly_degree] = {}
    rec_values[poly_degree] = {}
    for order in orders:
        prec_values[poly_degree][order] = {}
        rec_values[poly_degree][order] = {}
        
        X_train_pre, Y_train_pre, X_val_pre, Y_val_pre = split_train_val(Z, X, U, order=order, shuff=False, split_prop=1.0)
        X_train, Y_train = normalize_data_set(X_train_pre, Y_train_pre, X_train_pre)
        X_val, Y_val = normalize_data_set(X_train_pre, Y_train_pre, X_train_pre)

        #kernelize
        X_train_poly = polynomial_kernel_mat(X_train, poly_degree)
        X_val_poly = polynomial_kernel_mat(X_val, poly_degree)

        params = {'FEATURE_DIM': X_val_poly.shape[0], 'OUTPUT_DIM': Y_val_pre.shape[0]}
        net = linear_nnet(params)
        loss_func = nn.MSELoss()#SmoothL1Loss()
        optimizer = optim.SGD(net.parameters(),lr=0.01, momentum=0.9)
        epochs = 1000
        batch_size = 100

        for e in range(epochs):
            training_losses = []
            X_train_list, Y_train_list = minibatch_X_Y_arrays(X_train_poly, Y_train, batch_size)
            for i in enumerate(X_train_list):
                inp = Variable(torch.Tensor(X_train_list[i[0]].T))
                label = Variable(torch.Tensor(Y_train_list[i[0]].T))

                out = net(inp)
                optimizer.zero_grad()
                loss = loss_func(out, label)
                loss.backward()
                optimizer.step()

        inp_val = Variable(torch.Tensor(X_val_poly.T))
        label_val = Variable(torch.Tensor(Y_val.T))
        out_val = net(inp_val)
        loss_val = loss_func(out_val, label_val)
        mape_val = torch.mean(torch.abs(out_val - label_val)/label_val).item()

        A = copy.copy(net.l1.weight.data.numpy())
        

        Z, X, U = read_total_data_array("refbuild_med/seattle/RefBldgMediumOfficeNew2004_Chicago.eso", dep_var_names, ind_var_names, zone_temp_inds, first_diff=True, no_weekend_holiday=True)

        X_all_pre, Y_all_pre, X_v, Y_v = split_train_val(Z, X, U, order=order, shuff=False, split_prop=1.0)
        X_all, Y_all = normalize_data_set(X_all_pre, Y_all_pre, X_all_pre)

        #kernelize
        X_all_poly = polynomial_kernel_mat(X_all, poly_degree)
        
        Z_f, X_f, U_f = read_total_data_array("refbuild_med_airfault/seattle/Fault_FoulingAirFilter_RefBldgMediumOfficeNew2004.eso", dep_var_names, ind_var_names, zone_temp_inds, first_diff=True, no_weekend_holiday=True)

        X_all_pre_f, Y_all_pre_f, X_v_f, Y_v_f = split_train_val(Z_f, X_f, U_f, order=order, shuff=False, split_prop=1.0)
        X_all_f, Y_all_f = normalize_data_set(X_all_pre_f, Y_all_pre_f, X_all_pre)

        #kernelize
        X_all_poly_f = polynomial_kernel_mat(X_all_f, poly_degree)
        
        X = []
        Y = []

        for i in range(X_all_poly.shape[1]):
            if i % 1000 == 0:
                print("No fault data: ", np.around(100*(i/X_all_poly.shape[1])), "%")
            x = np.expand_dims(X_all_poly[:,i], axis=1)
            y = np.expand_dims(Y_all[:,i], axis=1)
            C = mat_C(x)
            D = mat_D(x, y, A)
            C_inv = np.linalg.inv(C)
            term1 = x.shape[0]*np.log(np.linalg.det(C_inv))
            term2 = np.trace(C_inv.dot(D.T).dot(D))
            term3 = -1.0*np.trace(A.dot(A.T))
            term4 = -2.0*np.trace(A.dot(x).dot(y.T))
            term5 = np.trace(x.T.dot(A.T).dot(A).dot(x))

            feat = [term1, term2, term3, term4, term5]
            features_app = np.asarray(feat)#(feat + list(x[:,0]) + list(y[:,0])) #np.expand_dims(A.flatten(), axis=1), x and y themselves seem to be a waste
            X.append(features_app)
            Y.append(0)

        for i in range(X_all_poly_f.shape[1]):
            if i % 1000 == 0:
                print("Fault data: ", np.around(100*(i/X_all_poly.shape[1])), "%")
            x = np.expand_dims(X_all_poly[:,i], axis=1) #tried with fault data
            y = np.expand_dims(Y_all_f[:,i], axis=1)
            C = mat_C(x)
            D = mat_D(x, y, A)
            C_inv = np.linalg.inv(C)
            term1 = x.shape[0]*np.log(np.linalg.det(C_inv))
            term2 = np.trace(C_inv.dot(D.T).dot(D))
            term3 = -1.0*np.trace(A.dot(A.T))
            term4 = -2.0*np.trace(A.dot(x).dot(y.T))
            term5 = np.trace(x.T.dot(A.T).dot(A).dot(x))

            feat = [term1, term2, term3, term4, term5]
            features_app = np.asarray(feat)#(feat + list(x[:,0]) + list(y[:,0])) #np.expand_dims(A.flatten(), axis=1)
            X.append(features_app)
            Y.append(1)

        X = np.asarray(X)
        Y = np.asarray(Y)
        Y = np.expand_dims(Y, axis=1)
        X = X.T
        Y = Y.T
        
        nofault_samps = []
        fault_samps = []

        for i in range(X.shape[1]):
            if Y[0,i] == 0:
                nofault_samps.append(X[:,i])
            else:
                fault_samps.append(X[:,i])
        
        for lag in lags:
            X_lag_nf = np.zeros((lag*X.shape[0], len(nofault_samps) - lag))
            Y_lag_nf = np.zeros((1, len(nofault_samps) - lag))

            X_lag_f = np.zeros((lag*X.shape[0], len(fault_samps) - lag))
            Y_lag_f = np.ones((1, len(fault_samps) - lag))

            for i in range(len(nofault_samps)-lag):
                samps = []
                for j in range(lag):
                    samps.append(nofault_samps[i + j])
                X_lag_nf[:,i] = np.asarray(samps).flatten()

            for i in range(len(fault_samps)-lag):
                samps = []
                for j in range(lag):
                    samps.append(fault_samps[i + j])
                X_lag_f[:,i] = np.asarray(samps).flatten()
            
            X = np.hstack((X_lag_nf, X_lag_f))
            Y = np.concatenate((Y_lag_nf, Y_lag_f), axis=1)
            
            train_vals = np.random.choice(X.shape[1], int(0.8*X.shape[1]), replace=False)
            vals = np.asarray([ i for i in range(X.shape[1]) if i not in train_vals ])

            X_train_log_reg = X[:,train_vals] #X[:,train_vals]
            Y_train_log_reg = Y[:,train_vals]

            X_val_log_reg = X[:,vals]
            Y_val_log_reg = Y[:,vals]
            
            logreg_model_obj = LogisticRegression(tol=0.0001, C=0.9, max_iter=10000, verbose=1)#, n_jobs=-1, solver="saga")
            logreg_model_obj.fit(X_train_log_reg.T, Y_train_log_reg.T)

            Y_hat = logreg_model_obj.predict(X_train_log_reg.T)

            p = sklearn.metrics.precision_score(Y_train_log_reg.T, Y_hat.T)
            r = sklearn.metrics.recall_score(Y_train_log_reg.T, Y_hat.T)

            print("Params: ", poly_degree, order, lag)
            
            print("Training precision: ", p)
            print("Training recall: ", r)

            Y_hat_val = logreg_model_obj.predict(X_val_log_reg.T)

            p = sklearn.metrics.precision_score(Y_val_log_reg.T, Y_hat_val.T)
            r = sklearn.metrics.recall_score(Y_val_log_reg.T, Y_hat_val.T)

            print("Validation precision: ", p)
            print("Validation recall: ", r)
            
            prec_values[poly_degree][order][lag] = p
            rec_values[poly_degree][order][lag] = r
            
            print("\n")